In [6]:
import os
os.chdir('/d/hpc/projects/FRI/DL/mo6643/MSC/cross_species_prediction/transVAE_seml/')
from transVAE.model.transVAE import transVAE
import scanpy as sc
import numpy as np
import gc
import pandas as pd
from gseapy import Biomart
import gseapy as gp
import logging

In [4]:
dataset_to_predict = "wang_to_predict_cleanCT.h5ad"
dataset_ground_truth1 = "dbdb_ground_truth_cleanCT.h5ad"
dataset_ground_truth2 = "mSTZ_ground_truth_cleanCT.h5ad"
addl_dataset_train = "train_adata_baseline.h5ad"
seed = 42
n_hidden = 1000 
n_latent = 512 
learning_rate = 0.0001 
cov_embed_dims = 10
max_epochs = 1
n_layers = 8
dropout_rate = 0.3
kl_weight = 0.0005
save_models = False
save_folder_name = ""
weight_decay = 0.0000001
shared_top = 4000
embed_ct = False

In [8]:
genes_intersection = pd.read_table("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/make_big_dataset/genes_shared_across_datasets.txt")
genes_intersection = genes_intersection.genes_shared_across_datasets.tolist()
categorical_covariate_embed_keys = ["dataset"]
# if embed variables
if embed_ct:
    categorical_covariate_embed_keys.append("cell_type")

logging.info(f"categorical_covariate_embed_keys values are: {categorical_covariate_embed_keys}")

if addl_dataset_train == "combination":
    datasets = ['train_adata_baseline.h5ad','extra_mouse_Embryonic.h5ad','extra_human_chem.h5ad','embedding_top100_mouse.h5ad','random_mouse_seed_42.h5ad',
              'embedding_top70_human.h5ad', 'extra_human_preT2D.h5ad','random_human_seed_42.h5ad','extra_human_neonatal.h5ad','extra_mouse_chem.h5ad',
              'extra_mouse_T1D.h5ad','extra_mouse_young.h5ad']

    genes_intersection = pd.read_table("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/make_big_dataset/genes_shared_across_datasets.txt")
    genes_intersection = genes_intersection.genes_shared_across_datasets.tolist()

    adatas = []
    for extra in datasets:
        adata = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/train_data/full_datasets/{extra}")
        adata = adata[:,adata.var.index.isin(genes_intersection)]
        adatas.append(adata)

    adata_train = sc.concat(adatas, join = "inner", axis = 0)
    adata_train = transVAE.setup_anndata(adata_train, categorical_covariate_embed_keys=categorical_covariate_embed_keys, categorical_covariate_keys=["organism"], copy = True)

    logging.info(f'Received the following configuration for datasets train: {addl_dataset_train}, shape: {adata_train.shape}')

elif addl_dataset_train == "train_adata_baseline_3000hvg.h5ad":
    # just the baseline data
    adata_train = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/train_data/full_datasets/train_adata_baseline.h5ad")
    logging.info(f'Received the following configuration for datasets train: {addl_dataset_train}, shape: {adata_train.shape}')
    adata_train = adata_train[:,adata_train.var.index.isin(genes_intersection)]
    adata_train = transVAE.setup_anndata(adata_train, categorical_covariate_embed_keys=categorical_covariate_embed_keys, categorical_covariate_keys=["organism"], copy = True)

else:
    adata_baseline = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/train_data/full_datasets/train_adata_baseline.h5ad")
    adata_extra = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/train_data/full_datasets/{addl_dataset_train}")
    adata_train = sc.concat([adata_baseline, adata_extra], join = "inner", axis = 0)
    logging.info(f'Received the following configuration for datasets train: {addl_dataset_train}, shape: {adata_train.shape}, baseline_shape: {adata_baseline.shape}')
    del adata_baseline
    del adata_extra
    gc.collect()
    ## making random splits
    adata_train = adata_train[:,adata_train.var.index.isin(genes_intersection)]
    adata_train = transVAE.setup_anndata(adata_train, categorical_covariate_embed_keys=categorical_covariate_embed_keys, categorical_covariate_keys=["organism"], copy = True)

# reading val data and data to translate
adata_to_predict = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/data_to_predict/{dataset_to_predict}")
dbdb_ground_truth = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/ground_truth/full_datasets/{dataset_ground_truth1}")
mSTZ_ground_truth = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/ground_truth/full_datasets/{dataset_ground_truth2}")

adata_to_predict = adata_to_predict[:,adata_to_predict.var.index.isin(genes_intersection)]
adata_to_predict = adata_to_predict[:,adata_to_predict.var.sort_index().index]
dbdb_ground_truth = dbdb_ground_truth[:,dbdb_ground_truth.var.index.isin(genes_intersection)]
mSTZ_ground_truth = mSTZ_ground_truth[:,mSTZ_ground_truth.var.index.isin(genes_intersection)]

logging.info(f'n_hidden: {n_hidden}, n_latent: {n_latent}, learning_rate: {learning_rate}, cov_embed_dims: {cov_embed_dims}, weight_decay {weight_decay}')
logging.info(f'max_epochs: {max_epochs}, n_layers: {n_layers}, dropout_rate: {dropout_rate}, kl_weight: {kl_weight}')

# check this data (full_datasets)

adata_to_predict_h = adata_to_predict[adata_to_predict.obs.disease == "healthy"]
adata_to_predict_OOD = adata_to_predict[adata_to_predict.obs.disease == "T2D"]
del adata_to_predict
gc.collect()

dbdb_ground_truth_h = dbdb_ground_truth[dbdb_ground_truth.obs.disease == "healthy"]
dbdb_ground_truth_OOD = dbdb_ground_truth[dbdb_ground_truth.obs.disease == "T2D"]
cts_to_consider = dbdb_ground_truth.obs.cell_type.unique()
del dbdb_ground_truth
gc.collect()

mSTZ_ground_truth_h = mSTZ_ground_truth[mSTZ_ground_truth.obs.disease == "healthy"]
mSTZ_ground_truth_OOD = mSTZ_ground_truth[mSTZ_ground_truth.obs.disease == "T2D"]
del mSTZ_ground_truth
gc.collect()

validation_adatas_dict = {"adata_to_predict":adata_to_predict_OOD,
                          "ground_truths": {"dbdb": dbdb_ground_truth_OOD,
                                            "mSTZ": mSTZ_ground_truth_OOD},
                          "translate_dicts": {"dbdb":{"dataset":"db/db", "organism": "Mus musculus"},
                                              "mSTZ":{"dataset":"mSTZ", "organism": "Mus musculus"}}}

# make the model
model = transVAE(adata_train, 
                 n_hidden=n_hidden, 
                 n_latent=n_latent, 
                 n_layers=n_layers, 
                 dropout_rate=dropout_rate, 
                 cov_embed_dims = cov_embed_dims, 
                 kl_weight = kl_weight,
                 validation_adatas_dict = validation_adatas_dict)

# train the model
model.train(batch_size=4096, max_epochs = max_epochs, train_size = 0.99, enable_progress_bar = True,
            early_stopping = True, early_stopping_monitor = 'mean_r2_validation_eval', early_stopping_mode = "max", 
            early_stopping_min_delta = 0.01, early_stopping_patience = 70,
            plan_kwargs = {"lr":learning_rate,
                           "weight_decay":weight_decay,
                           "reduce_lr_on_plateau":True,
                           "lr_factor":0.5,
                           "lr_patience":50,
                           "lr_scheduler_metric":"reconstruction_loss_validation"})

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


INFO     Using column names from columns of adata.obsm['covariates']                                               
INFO     Using column names from columns of adata.obsm['covariates_embed']                                         


/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value


INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
SLURM auto-requeueing enabled. Setting signal handlers.


Epoch 1/1: 100%|██████████| 1/1 [00:45<00:00, 45.52s/it, v_num=1, train_loss_step=992, train_loss_epoch=2.24e+3]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 1/1: 100%|██████████| 1/1 [00:45<00:00, 45.52s/it, v_num=1, train_loss_step=992, train_loss_epoch=2.24e+3]


In [9]:
# dbdb
preds_dbdb_h = model.translate(adata_to_predict_h, translate_dict={"dataset":"db/db", "organism": "Mus musculus"})
preds_dbdb_OOD = model.translate(adata_to_predict_OOD, translate_dict={"dataset":"db/db", "organism": "Mus musculus"})

# mSTZ
preds_mSTZ_h = model.translate(adata_to_predict_h, translate_dict={"dataset":"mSTZ", "organism": "Mus musculus"})
preds_mSTZ_OOD = model.translate(adata_to_predict_OOD, translate_dict={"dataset":"mSTZ", "organism": "Mus musculus"})

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value


INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


In [55]:
# GSEA performance -> TODO should be done on just on B cell from pancereas
# load the biomart m2h translation
m2h = pd.read_csv("./final_experiments/mouse2human_translations_for_GSEA_on-the-fly.csv")

ct = "type B pancreatic cell"
# GT
dbdb_ground_truth_h_Bcells = dbdb_ground_truth_h[dbdb_ground_truth_h.obs.cell_type == ct]
mSTZ_ground_truth_h_Bcells = mSTZ_ground_truth_h[mSTZ_ground_truth_h.obs.cell_type == ct]
# preds
preds_dbdb_OOD_Bcells = preds_dbdb_OOD[preds_dbdb_OOD.obs.cell_type == ct]
preds_mSTZ_OOD_Bcells = preds_mSTZ_OOD[preds_mSTZ_OOD.obs.cell_type == ct]

# dbdb GT -> clean for GSEA
new_var = dbdb_ground_truth_h_Bcells.var.merge(m2h, left_index=True, right_on="ensembl_gene_id", how = "left")
new_var.index = new_var["hsapiens_homolog_associated_gene_name"]
dbdb_ground_truth_h_Bcells.var = new_var

# mSTZ GT -> clean for GSEA
new_var = mSTZ_ground_truth_h_Bcells.var.merge(m2h, left_index=True, right_on="ensembl_gene_id", how = "left")
new_var.index = new_var["hsapiens_homolog_associated_gene_name"]
mSTZ_ground_truth_h_Bcells.var = new_var

# dbdb preds -> clean for GSEA
new_var = preds_dbdb_OOD_Bcells.var.merge(m2h, left_index=True, right_on="ensembl_gene_id", how = "left")
new_var.index = new_var["hsapiens_homolog_associated_gene_name"]
preds_dbdb_OOD_Bcells.var = new_var

# mSTZ preds -> clean for GSEA
new_var = preds_mSTZ_OOD_Bcells.var.merge(m2h, left_index=True, right_on="ensembl_gene_id", how = "left")
new_var.index = new_var["hsapiens_homolog_associated_gene_name"]
preds_mSTZ_OOD_Bcells.var = new_var

# GSEA dbdb
dbdb = sc.concat([dbdb_ground_truth_h_Bcells, preds_dbdb_OOD_Bcells])
mSTZ = sc.concat([mSTZ_ground_truth_h_Bcells, preds_mSTZ_OOD_Bcells])

res_KEGG =  gp.gsea(data=dbdb.to_df().T, # row -> genes, column-> samples
                    gene_sets="KEGG_2019_Mouse",
                    cls=dbdb.obs.disease,
                    permutation_num=1000,
                    permutation_type='phenotype',
                    outdir=None,
                    method='s2n', # signal_to_noise
                    threads=128,
                    verbose=True)

GSEA_results_dbdb = res_KEGG.res2d

res_KEGG =  gp.gsea(data=mSTZ.to_df().T, # row -> genes, column-> samples
                    gene_sets="KEGG_2019_Mouse",
                    cls=mSTZ.obs.disease,
                    permutation_num=1000,
                    permutation_type='phenotype',
                    outdir=None,
                    method='s2n', # signal_to_noise
                    threads=128,
                    verbose=True)

GSEA_results_mSTZ = res_KEGG.res2d

# get GT
gt_pathways = pd.read_csv("gt_top_100_pathways.csv")
gt_top100pathways_dbdb = set(gt_pathways.dbdb_top100_pathways.tolist())
gt_top100pathways_mSTZ = set(gt_pathways.mSTZ_top100_pathways.tolist())

# compare
GSEA_results_dbdb_pathways = set(GSEA_results_dbdb.Term[1:100].tolist())
GSEA_results_mSTZ_pathways = set(GSEA_results_mSTZ.Term[1:100].tolist())

len(gt_top100pathways_dbdb.intersection(GSEA_results_dbdb_pathways))
len(gt_top100pathways_mSTZ.intersection(GSEA_results_mSTZ_pathways))

### Making GT 100 top pathways

In [61]:
dbdb_ground_truth = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/ground_truth/full_datasets/{dataset_ground_truth1}")
mSTZ_ground_truth = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/ground_truth/full_datasets/{dataset_ground_truth2}")

In [70]:
dbdb_ground_truth = dbdb_ground_truth[:,dbdb_ground_truth.var.index.isin(genes_intersection)]
mSTZ_ground_truth = mSTZ_ground_truth[:,mSTZ_ground_truth.var.index.isin(genes_intersection)]

In [71]:
dbdb_GT_Bcells = dbdb_ground_truth[dbdb_ground_truth.obs.cell_type == ct]
mSTZ_GT_Bcells = mSTZ_ground_truth[mSTZ_ground_truth.obs.cell_type == ct]

In [78]:
# dbdb GT -> clean for GSEA
new_var = dbdb_GT_Bcells.var.merge(m2h, left_index=True, right_on="ensembl_gene_id", how = "left")
new_var.index = new_var["hsapiens_homolog_associated_gene_name"]
dbdb_GT_Bcells.var = new_var

# mSTZ GT -> clean for GSEA
new_var = mSTZ_GT_Bcells.var.merge(m2h, left_index=True, right_on="ensembl_gene_id", how = "left")
new_var.index = new_var["hsapiens_homolog_associated_gene_name"]
mSTZ_GT_Bcells.var = new_var

In [90]:
res_KEGG =  gp.gsea(data=dbdb_GT_Bcells.to_df().T, # row -> genes, column-> samples
                    gene_sets="KEGG_2019_Mouse",
                    cls=dbdb_GT_Bcells.obs.disease,
                    permutation_num=1000,
                    permutation_type='phenotype',
                    outdir=None,
                    method='s2n', # signal_to_noise
                    threads=128,
                    verbose=True)

GSEA_results_dbdb_GT = res_KEGG.res2d

2023-12-17 15:25:21,640 [INFO] Parsing data files for GSEA.............................
2023-12-17 15:25:25,160 [INFO] Enrichr library gene sets already downloaded in: /d/hpc/home/mo6643/.cache/gseapy, use local file
2023-12-17 15:25:25,179 [INFO] 0025 gene_sets have been filtered out when max_size=500 and min_size=15
2023-12-17 15:25:25,180 [INFO] 0278 gene_sets used for further statistical testing.....
2023-12-17 15:25:25,181 [INFO] Start to run GSEA...Might take a while..................
2023-12-17 15:27:59,351 [INFO] Congratulations. GSEApy ran successfully.................



In [88]:
res_KEGG =  gp.gsea(data=mSTZ_GT_Bcells.to_df().T, # row -> genes, column-> samples
                    gene_sets="KEGG_2019_Mouse",
                    cls=mSTZ_GT_Bcells.obs.disease,
                    permutation_num=1000,
                    permutation_type='phenotype',
                    outdir=None,
                    method='s2n', # signal_to_noise
                    threads=128,
                    verbose=True)

GSEA_results_mSTZ_GT = res_KEGG.res2d

2023-12-17 15:23:57,260 [INFO] Parsing data files for GSEA.............................
2023-12-17 15:23:58,964 [INFO] Enrichr library gene sets already downloaded in: /d/hpc/home/mo6643/.cache/gseapy, use local file
2023-12-17 15:23:58,978 [INFO] 0026 gene_sets have been filtered out when max_size=500 and min_size=15
2023-12-17 15:23:58,980 [INFO] 0277 gene_sets used for further statistical testing.....
2023-12-17 15:23:58,980 [INFO] Start to run GSEA...Might take a while..................
2023-12-17 15:25:20,716 [INFO] Congratulations. GSEApy ran successfully.................



In [96]:
gt_pathways = pd.DataFrame({"dbdb_top100_pathways":GSEA_results_dbdb_GT.Term[0:100], "mSTZ_top100_pathways":GSEA_results_mSTZ_GT.Term[0:100]})

In [119]:
gt_pathways.to_csv("gt_top_100_pathways.csv", index_label="", index = False)

In [134]:
GSEA_results_mSTZ_GT

,Name,Term,ES,NES,NOM p-val,FDR q-val,FWER p-val,Tag %,Gene %,Lead_genes
0,gsea,Protein processing in endoplasmic reticulum,-0.648912,-2.721918,0.0,0.0,0.0,66/136,14.01%,SSR4;SEC61B;PDIA6;ERLEC1;EDEM2;DNAJC3;SSR2;DAD...
1,gsea,Osteoclast differentiation,0.552408,2.493151,0.0,0.0,0.0,27/95,12.84%,FOSB;FOS;JUN;JUNB;FOSL2;NFKBIA;JUND;SOCS3;FOSL...
2,gsea,Ribosome,-0.747836,-2.424443,0.0,0.0,0.0,50/86,11.90%,RPL4;RPL13;RPS5;RPL7;RPL6;RPS19;RPL3;RPS7;RPS1...
3,gsea,Glycolysis / Gluconeogenesis,-0.634025,-2.403574,0.0,0.0,0.0,12/45,10.47%,ALDH1A3;ALDOB;FBP2;PGK1;AKR1A1;FBP1;BPGM;PKM;G...
4,gsea,Protein export,-0.839893,-2.391023,0.0,0.0,0.0,12/19,9.60%,SEC11C;SEC61B;SRP9;SPCS1;SRP14;SEC11A;SPCS2;SR...
...,...,...,...,...,...,...,...,...,...,...
272,gsea,Notch signaling pathway,-0.194754,-0.733477,0.873541,0.92279,1.0,24/40,56.98%,NCSTN;CTBP1;APH1A;HES1;MAML2;JAG2;RFNG;NUMBL;S...
273,gsea,Steroid hormone biosynthesis,-0.24154,-0.726642,0.834829,0.924598,1.0,10/18,47.92%,HSD17B12;HSD17B7;SULT2B1;CYP19A1;SRD5A1;CYP1B1...
274,gsea,Nicotinate and nicotinamide metabolism,-0.208715,-0.676999,0.929236,0.958542,1.0,12/22,49.94%,NADK;NT5M;NT5E;NT5C3B;ENPP1;BST1;NUDT12;NMNAT1...
275,gsea,Aminoacyl-tRNA biosynthesis,0.195594,0.63381,0.930902,0.974531,1.0,21/21,81.71%,SEPSECS;NARS2;LARS2;QRSL1;PARS2;AARS2;RARS2;IA...


### How many do I expect to match by chance?

In [137]:
(100/277)*100

36.101083032490976